# Moving Average
This strategy does not look at the current price but instead at the moving average of the price pa. We will consider different time intervals for calculating the moving average: two hours, six hours, one day, seven days, 30 days, 100 days, and 200 days. The liquidity position of this strategy will be [pa − a, pa + a]. As in the constant interval strategy 4.3, the leftover token will not be used to provide any liquidity.

Import code dependencies

In [1]:
import math
from datetime import date, timedelta
from typing import Dict
from _decimal import Decimal

import pandas as pd

from demeter import TokenInfo, Actuator, Strategy, RowData, simple_moving_average, ChainType, MarketInfo, PeriodTrigger, BaseAction, EvaluatorEnum
from demeter.uniswap import UniV3Pool, UniLpMarket

Set pandas output format

In [2]:
pd.options.display.max_columns = None
pd.set_option("display.width", 5000)

Custom Moving Average strategy with code to add liquidity at window's average price.

In [3]:
class MovingAverageStrategy(Strategy):

    def initialize(self):
        """
        This function is called before main loop is executed.
        you can prepare data, or register trigger here
        """

        # Add a simple moving average line for backtesting data. In backtesting,
        # we will add/remove liquidity according to this line.
        self.add_column(market_key, "sma", simple_moving_average(self.data[market_key].price, window=timedelta(hours=1)))
        self.net_value_before_bar = 0
        self.net_value_diff_list = []
        # Register a trigger, every day, we split both assets into two shares of equal value
        self.triggers.append(PeriodTrigger(time_delta=timedelta(days=1), trigger_immediately=True, do=self.rebalance))

    def rebalance(self, row_data: RowData):
        self.markets[market_key].even_rebalance(row_data.market_status[market_key].price)

    """
    After a test is executed, actuator will loop the data, and bar series functions will be called on every time.
    Here you can set conditions and execute liquidity operations
    """

    def on_bar(self, row_data: RowData):
        """
        This function is called after trigger, but before market is updated(Fees will be distributed in this step).
        """
        lp_market: UniLpMarket = self.markets[market_key]
        current_price = row_data.market_status[market_key].price
        # get moving average price, if value is nan, fill it with current price
        ma_price = self.data[market_key].loc[row_data.timestamp]["sma"]
        ma_price = row_data.market_status[market_key].price if math.isnan(ma_price) else ma_price

        # this is a nonsense strategy, just to show how to trigger actions
        if row_data.market_status[market_key].price > ma_price + 25 and len(self.markets[market_key].positions) < 1:
            lp_market.remove_all_liquidity()
            lp_market.add_liquidity(current_price, current_price + 100)
        elif row_data.market_status[market_key].price < ma_price - 25 and len(self.markets[market_key].positions) < 1:
            lp_market.remove_all_liquidity()
            lp_market.add_liquidity(current_price - 100, current_price)

    def after_bar(self, row_data: RowData):
        """
        this function is called after market has updated.
        """
        timestamp = row_data.timestamp
        net_value_after_bar = self.broker.get_account_status(self.prices.loc[timestamp]).net_value
        net_value_diff = net_value_after_bar - self.net_value_before_bar
        self.net_value_diff_list.append(net_value_diff)

    def finalize(self):
        """
        Call when main loop finished. some statistic work can be executed here
        """
        self.data[market_key]["diff"] = self.net_value_diff_list
        pass

    def notify(self, action: BaseAction):
        """
        When a new action(add/remove liquidity) is executed, you can be notified by this call back.
        """
        print("\n")
        print(action.timestamp, action.action_type.value)

Main logic to run Actuator, init two token and market with key "market1"

In [4]:
usdc = TokenInfo(name="usdc", decimal=6)  # TokenInfo(name='usdc', decimal=6)
eth = TokenInfo(name="eth", decimal=18)  # TokenInfo(name='eth', decimal=18)
pool = UniV3Pool(
    usdc, eth, 0.05, usdc
)  # PoolBaseInfo(Token0: TokenInfo(name='usdc', decimal=6),Token1: TokenInfo(name='eth', decimal=18),fee: 0.0500,base token: usdc)

market_key = MarketInfo("market1")  # market1
market = UniLpMarket(market_key, pool)
market.data_path = "../data"
market.load_data(ChainType.polygon.name, "0x45dda9cb7c25131df268515131f647d726f50608", date(2023, 8, 15), date(2023, 8, 15))

actuator = Actuator()  # init actuator
actuator.broker.add_market(market)  # add market to actuator
actuator.broker.set_balance(usdc, 10000)  # set balance
actuator.broker.set_balance(eth, 10)  # set balance
actuator.strategy = MovingAverageStrategy()  # set strategy
actuator.set_price(market.get_price_from_data())  # set actuator price

# actuator.run()  # run actuator

Run actuator with evaluators and save result to files

In [5]:
actuator.run(evaluator=[
    EvaluatorEnum.max_draw_down,
    EvaluatorEnum.annualized_returns,
    EvaluatorEnum.net_value,
    EvaluatorEnum.profit,
    EvaluatorEnum.net_value_up_down_rate,
    EvaluatorEnum.eth_up_down_rate,
    EvaluatorEnum.position_fee_profit,
    EvaluatorEnum.position_fee_annualized_returns,
    EvaluatorEnum.position_market_time_rate,
])
evaluating_result: Dict[EvaluatorEnum, Decimal] = actuator.evaluating_indicator

actuator.save_result(
    path="./result",  # save path
    account=True,  # save account status list as a csv file
    actions=True,  # save actions as a json file and a pickle file
)

2023-11-03 10:18:17,432 - INFO - init strategy...
2023-11-03 10:18:17,437 - INFO - start main loop...
 13%|██████████████▎                                                                                              | 189/1440 [00:00<00:01, 967.18it/s]



2023-08-15 00:00:00 sell


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1440/1440 [00:01<00:00, 998.43it/s]
2023-11-03 10:18:18,915 - INFO - main loop finished
2023-11-03 10:18:18,929 - INFO - Start calculate evaluating indicator...
/Users/florije/PycharmProjects/demeter/demeter/core/evaluating_indicator.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fee_df.sort_values(by=["market1_base_uncollected", "market1_quote_uncollected"], ascending=[False, False], inplace=True)
2023-11-03 10:18:18,946 - INFO - Evaluating indicator has finished it's job.
2023-11-03 10:18:18,952 - INFO - Print actuator summary
2023-11-03 10:18:18,963 - INFO - Backtesting finished, execute time 1.531486988067627s
2023-11-03 10:18:18,980 - INFO - files have saved to ./result/backtest

Final account status                              
Token balance in broker       
USDC      :14216.36285789554574593764472ETH       :7.711946160408465926967320488
Position value in markets     
market1(UniLpMarket)
token0    :USDC                     token1    :ETH                      fee       :0.0500                   is 0 base :True                     
positions 
Empty DataFrame


Account balance history                           
                                         net_value                           USDC                            ETH market1_net_value market1_base_uncollected market1_quote_uncollected market1_base_in_position market1_quote_in_position  market1_position_count
2023-08-15 00:00:00  28434.83495183806275037750938  14216.36285789554574593764472  7.711946160408465926967320488             0E-24                        0                         0                        0                         0                       0
2023-08-15 00:01:00  28434.8349518380627503775

['./result/backtest-20231103-101818.account.csv',
 './result/backtest-20231103-101818.action.json',
 './result/backtest-20231103-101818.action.pkl']